In [1]:
import os
import pandas as pd
from pathlib import Path
import numpy as np

In [2]:
# from Bio import SeqIO
# from Bio.Seq import Seq  # Import the Seq class
# from Bio.SeqRecord import SeqRecord  # Import the SeqRecord class for creating sequence records
# from collections import defaultdict

In [3]:
proj_dir="/master/nplatt/schisto_aim_panel"
results_dir=f"{proj_dir}/results"

In [4]:
info_df=pd.read_csv(f"{proj_dir}/data/platt_et_al_2025_stable1.csv", sep=",", header=0)
info_df.head()

,Library ID,Museum Accession Number (NHM) or Donor ID,Predicted Species,Mitchondrial Haplotype,Population Assignment,NCBI SRA Accession,Project Citation,Country,Locality,Latitude,...,Original life-cycle stage collected,Life-cycle stage sequenced,Num Read Pairs (1e6),Coverage,Final SNV Dataset,Discordant COX1/ITS,% S. haematobium ancestry (q),Origin,Project (Collector),Comments
0,angola_cota_57,MCF03050E0612,S. haematobium,S. haematobium,S. haematobium (southern),SRR11907442,Herein,Angola,Source of the Cota river,-9.26186,...,cercariae,cercariae,76.6,32.96,True,NaN,100.00%,SCAN,Angola malacology survey 2013- NNTDCP/CIS and ...,NaN
1,angola_cota_58,MCF03050E0613,S. haematobium,S. haematobium,S. haematobium (southern),SRR11907441,Herein,Angola,Source of the Cota river,-9.26186,...,cercariae,cercariae,71.6,32.83,True,NaN,100.00%,SCAN,Angola malacology survey 2013- NNTDCP/CIS and ...,NaN
2,angola_cota_59,MCF03050E0614,S. haematobium,S. haematobium,S. haematobium (southern),SRR11907440,Herein,Angola,Source of the Cota river,-9.26186,...,cercariae,cercariae,66.5,25.61,True,NaN,100.00%,SCAN,Angola malacology survey 2013- NNTDCP/CIS and ...,NaN
3,angola_icau_60,MCF03050E0615,S. haematobium,S. haematobium,S. haematobium (southern),SRR11907439,Herein,Angola,Icau Wando village,-8.6451,...,miracidia,miracidia,56.7,25.73,True,NaN,100.00%,SCAN,Angola malacology survey 2013- NNTDCP/CIS and ...,NaN
4,angola_icau_61,MCF03050E0615,S. haematobium,S. haematobium,S. haematobium (southern),SRR11907438,Herein,Angola,Icau Wando village,-8.6451,...,miracidia,miracidia,75.3,30.40,True,NaN,100.00%,SCAN,Angola malacology survey 2013- NNTDCP/CIS and ...,NaN


# Get raw VCF data

In [5]:
os.chdir(f"{proj_dir}/data")

In [ ]:
!cp ~/sch_hae_scan/results/filter_genotypes/sorted_annotated_snps.vcf scan_snvs.vcf

# LD Filter the VCF file

In [8]:
Path(f"{results_dir}/split_data_vcf").mkdir(parents=True, exist_ok=True)
os.chdir(f"{results_dir}/split_data_vcf")

In [ ]:
%%bash

conda run -n popgen --live-stream \
    plink2 \
        --vcf ../../data/scan_snvs.vcf \
        --allow-extra-chr \
        --double-id \
        --indep-pairwise 25 5 0.20 \
        --out ld

PLINK v2.00a5.12LM 64-bit Intel (25 Jun 2024)  www.cog-genomics.org/plink/2.0/
urcell, Christopher Chang   GNU General Public License v3
Logging to ld.log.
Options in effect:
allow-extra-chr
  --double-id
  --indep-pairwise 25 5 0.20
  --out ld
cf--vcf ../../data/scan_snvs.v

Start time: Wed Mar  5 15:13:09 2025
643 MiB for maindetected, ~1016614 available; reserving 515
workspace.
Using up to 192 threads (change this with --threads).
--vcf: 3456k variants scanned.

In [ ]:
%%bash

conda run -n vcftools --live-stream \
    vcftools \
        --vcf ../../data/scan_snvs.vcf \
        --exclude ld.prune.out \
        --recode \
        --recode-INFO-all \
        --stdout \
        >scan_snvs.ld.vcf

# Reduce the number of samples to a development and test data sets

In [ ]:
info_df.loc[ info_df["Final SNV Dataset"] == True]["Population Assignment"].value_counts()



I want to randomly select 50 northern samples, 50 southern samples and 15 S. bovis for the "panel" and then save the remaining to be in the test



In [ ]:
# Define the sample sizes for each population
sample_sizes = {"S. haematobium (northern)": 50, 
                "S. haematobium (southern)": 50, 
                "S. bovis": 15}

# Sample the specified number of individuals from each population
panel_df = pd.concat(
    [info_df[info_df["Population Assignment"] == pop].sample(n=size, random_state=42)
     for pop, size in sample_sizes.items() if pop in info_df["Population Assignment"].values]
)

panel_df.reset_index(drop=True, inplace=True)
panel_df["Library ID"].to_csv("panel.samples.list", sep=",", header=False, index=False)
panel_df.to_csv("panel.df.csv", sep=",", header=True, index=False)
panel_df

# sample these individuals from the VCF file

In [ ]:
%%bash

conda run -n popgen --live-stream vcftools --vcf scan_snvs.ld.vcf --not-chr NC_067195.1 --keep panel.samples.list --recode --recode-INFO-all --stdout >panel.vcf

In [ ]:
%%bash

conda run -n popgen --live-stream vcftools --vcf scan_snvs.ld.vcf --not-chr NC_067195.1 --remove panel.samples.list --recode --recode-INFO-all --stdout >test.vcf

# Fst

In [ ]:
Path(f"{results_dir}/fst").mkdir(parents=True, exist_ok=True)
os.chdir(f"{results_dir}/fst")

In [ ]:
panel_df=pd.read_csv(f"{results_dir}/split_data_vcf/panel.df.csv", sep=",", header=0)

In [ ]:
panel_df.loc[panel_df["Predicted Species"] == 'S. haematobium', "Library ID"].to_csv("shae.panel.list", sep=",", header=False, index=False)
panel_df.loc[panel_df["Predicted Species"] == 'S. bovis', "Library ID"].to_csv("sbov.panel.list", sep=",", header=False, index=False)

In [ ]:
%%bash

conda run -n popgen --live-stream \
    vcftools \
        --vcf ../split_data_vcf/panel.vcf \
        --weir-fst-pop shae.panel.list \
        --weir-fst-pop sbov.panel.list

In [ ]:
fst_df=pd.read_csv("out.weir.fst", sep="\t", header=0)
fst_df

In [ ]:
major_chroms= [f"NC_067{x}.1" for x in list(range(196, 203))]
major_chroms

In [ ]:
fst_df = fst_df.loc[((fst_df["WEIR_AND_COCKERHAM_FST"]>0) & fst_df["CHROM"].isin(major_chroms))].reset_index(drop=True)
fst_df.to_csv("fst_df.csv", sep=",", header=True, index=False)
fst_df

# PCA

In [ ]:
Path(f"{results_dir}/pca").mkdir(parents=True, exist_ok=True)
os.chdir(f"{results_dir}/pca")

In [ ]:
info_df=pd.read_csv(f"{proj_dir}/data/platt_et_al_2025_stable1.csv", sep=",", header=0)
panel_df=pd.read_csv(f"{results_dir}/split_data_vcf/panel.df.csv", sep=",", header=0)
panel_df

In [ ]:
%%bash
conda run -n popgen --cwd . --live-stream\
    plink2 \
        --threads 12 \
        --vcf ../split_data_vcf/panel.vcf \
        --pca 20 allele-wts \
        --double-id \
        --allow-extra-chr \
        --out pca

In [ ]:
# %%bash
# conda run -n popgen --cwd . --live-stream\
#     plink2 \
#         --threads 12 \
#         --vcf ../split_data_vcf/panel.vcf \
#         --pca 20 allele-wts \
#         --double-id \
#         --allow-extra-chr \
#         --out pca

In [ ]:
#get eigen values and sample labels
pca_df=pd.read_csv("pca.eigenvec", sep="\t", header=0)
pca_df=pca_df.drop("#FID", axis=1)

#merge the dataframes
pca_df=pca_df.merge(panel_df, how='left', left_on="IID", right_on='Library ID')

#fix the country which contains some float NaNs
pca_df["country"] = pca_df["Country"].astype(str)

countries = sorted(pca_df["country"].unique().astype(str))
pca_df

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import yaml

# Load the YAML configuration file
with open(f"{proj_dir}/code/plotting_config.yaml", "r") as config_file:
    config = yaml.safe_load(config_file)

# Create a multipanel figure
fig, axes = plt.subplots(1, 2, figsize=(12, 6))  # 1 row, 2 columns

# Panel 1: PC1 vs PC2
for index, row in pca_df.iterrows():
    x = row["PC1"]
    y = row["PC2"]
    species = row["Predicted Species"]
    color = config["species"][species]["color"]
    
    # Plot PC1 vs PC2
    axes[0].scatter(x, y, color=color, edgecolor="black", s=50, linewidths=0.5)

axes[0].set_xlabel("PC1")
axes[0].set_ylabel("PC2")
axes[0].set_title("PC1 vs PC2")


# # Add the legend to the figure (shared across panels)
# fig.legend(
#     handles=species_legend, 
#     title="Species", 
#     loc="center right", 
#     bbox_to_anchor=(1.1, 0.5)
# )

# # Adjust layout to ensure everything fits
# plt.tight_layout()

# # Optional: Save the plot
# # plt.savefig('pca.png', facecolor="white", dpi=600, bbox_inches='tight')
# # plt.savefig('pca.svg')

# # Display the figure
# plt.show()


In [ ]:
pca_df.loc[pca_df["PC2"]<-0.2]

In [ ]:
info_df.loc[info_df["Country"] == "Uganda"]

# InfoCalc

In [ ]:
# VENN diagram/overlap of the 5 datasets